In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
import os

In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi

# Data

In [10]:
api = KaggleApi()
api.authenticate()

In [12]:
dataset_name = 'yashpaloswal/ann-car-sales-price-prediction'
download_path = './db'
api.dataset_download_files(dataset_name, path=download_path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/yashpaloswal/ann-car-sales-price-prediction


In [16]:
os.listdir(download_path)

['car_purchasing.csv', 'c10']